In [ ]:
%matplotlib inline
import cv2
cap = cv2.VideoCapture(r"C:\Users\bsriram\Desktop\Data\ACM_Data\AllData\SI_sample\behavCam1.avi")
cap.set(1,300)
success,image = cap.read()
image = cv2.arrowedLine(image,(0,0),(100,100),(0.,0.,255.),3)  
cv2.imshow('ImageWindow', image)
cv2.waitKey()

fps = 1/cap.get(2)
cap.release()
print(fps)

# analyze a specific session

In [ ]:
%matplotlib inline
import cv2
import os
import numpy as np
import tqdm
import collections
import matplotlib.pyplot as plt
from dlc_utils import get_all_videos_in_session,get_analysis_dfs,get_body_length
from nor_si_utils import get_object_locations
import sys

session = r'C:\Users\bsriram\Desktop\Data\ACM_Data\AllData\SI_sample'

vids = get_all_videos_in_session(session)
df = get_analysis_dfs(session,vids)
df = get_body_length(df)
# fig,ax=plt.subplots(1,1,figsize=(10,10))
# ax.plot(df['body_length'])
bl_m = df['body_length'].mean()
bl_sd = df['body_length'].std()

bl_obj,tr_obj = get_object_locations(df)

# ax.plot(ax.get_xlim(),[bl_m,bl_m],'k--')
# ax.plot(ax.get_xlim(),[bl_m-bl_sd,bl_m-bl_sd],'k--')
# ax.plot(ax.get_xlim(),[bl_m+bl_sd,bl_m+bl_sd],'k--')
try:
    curr_filename = os.path.join(session,vids[0]+'.avi')
    cap = cv2.VideoCapture(curr_filename)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    writer = cv2.VideoWriter(r"C:\Users\bsriram\Desktop\Data\ACM_Data\AllData\annotated_SI_part.avi",fourcc, 1/cap.get(2), 
                             (int(cap.get(3)),int(cap.get(4)+100)))
    body_len = collections.deque([np.nan],maxlen=300)
    for idx,row in tqdm.tqdm(df.iterrows(),total=df.shape[0]):
        if idx<900 or idx>1900 : continue
        if row['source'][0] in curr_filename:
            # cv is open to the right place
            pass
        else:
            cap.release()
            curr_filename = os.path.join(session,row['source'][0]+'.avi')
            cap=cv2.VideoCapture(curr_filename)
        # get the frame
        cap.set(1,row['frame_number'][0]) # 1==set frame number
        succ,img = cap.read()
        #annotate
        arrow_col = (0.,0.,255.)
        arrow_size = 3
        if not np.isnan(row['body_length'][0]):
            try:
                img = cv2.arrowedLine(img,row['bl_arrow_start'][0],row['bl_arrow_end'][0],arrow_col,arrow_size)
            except ValueError as er:
                # ok found a nan value - skipping
                pass
        
        
        # modify image
        new_img = np.zeros((img.shape[0]+100,img.shape[1],img.shape[2])).astype(np.uint8)
        new_img[:img.shape[0],:,:] += img[:,:,:]
        
        # draw the object circles
        new_img = cv2.circle(new_img,bl_obj,5,(0,255,0))
        new_img = cv2.circle(new_img,tr_obj,5,(255,0,0))
        
        # body length calculations
        body_len.append(row['body_length'][0])
        new_img = cv2.line(new_img,(10,475),(310,475),(127,127,127),1)
        new_img = cv2.line(new_img,(10,475),(10,425),(127,127,127),1)
        for i,length in enumerate(body_len):
            if not np.isnan(length):
                if length>bl_m-bl_sd: new_img = cv2.circle(new_img,(10+i,int(475-1.5*length)),1,(255,255,255))
                else: new_img = cv2.circle(new_img,(10+i,int(475-1.5*length)),3,(255,0,0))
        
        # add to writer
        writer.write(new_img)
except Exception as er:
    import pdb
    pdb.set_trace()
    
    raise er.with_traceback(sys.exc_info()[2])
finally:
    # for index, row in df.iterrows():
    cap.release()
    writer.release()
    cv2.destroyAllWindows()

In [ ]:
%matplotlib inline
import cv2
import os
import numpy as np
import tqdm
import collections
import matplotlib.pyplot as plt
from dlc_utils import get_all_videos_in_session,get_analysis_dfs,get_body_length
from ezm_utils import get_quadrant
from nor_si_utils import get_object_locations
import sys

session = r'Y:\Data 2018-2019\Complement 4 - schizophrenia Project\2019 Adult Behavior C4_for revisions\EZM\ALC_060519_2_mC4\ALC_060519_2_57G_EZM'

vids = get_all_videos_in_session(session)
df = get_analysis_dfs(session,vids)
location, body_loc, zm_center, open_coord, closed_coord = get_quadrant(df)


try:
    curr_filename = os.path.join(session,vids[0]+'.avi')
    cap = cv2.VideoCapture(curr_filename)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    writer = cv2.VideoWriter(r"C:\Users\User\Desktop\Code\annotated_SI_part.avi",fourcc, 1/cap.get(2), 
                             (int(cap.get(3)),int(cap.get(4))))
    for idx,row in tqdm.tqdm(df.iterrows(),total=df.shape[0]):
#         if idx<900 or idx>1900: continue
        if not idx%10==0:continue
        if row['source'][0] in curr_filename:
            # cv is open to the right place
            pass
        else:
            cap.release()
            curr_filename = os.path.join(session,row['source'][0]+'.avi')
            cap=cv2.VideoCapture(curr_filename)
        # get the frame
        cap.set(1,row['frame_number'][0]) # 1==set frame number
        succ,img = cap.read()
        
        # draw the object circles
        img = cv2.circle(img,(int(zm_center[0]),int(zm_center[1])),5,(0,0,255))
        img = cv2.circle(img,(int(open_coord[0]),int(open_coord[1])),5,(0,0,255))
        img = cv2.circle(img,(int(closed_coord[0]),int(closed_coord[1])),5,(0,0,255))
        
        if not np.isnan(location[idx]):
            if location[idx]==1:
                img = cv2.circle(img,(int(body_loc[0][idx]),int(body_loc[1][idx])),5,(255,0,0))
            else:
                img = cv2.circle(img,(int(body_loc[0][idx]),int(body_loc[1][idx])),5,(0,255,0))
        
#         # body length calculations
#         body_len.append(row['body_length'][0])
#         new_img = cv2.line(new_img,(10,475),(310,475),(127,127,127),1)
#         new_img = cv2.line(new_img,(10,475),(10,425),(127,127,127),1)
#         for i,length in enumerate(body_len):
#             if not np.isnan(length):
#                 if length>bl_m-bl_sd: new_img = cv2.circle(new_img,(10+i,int(475-1.5*length)),1,(255,255,255))
#                 else: new_img = cv2.circle(new_img,(10+i,int(475-1.5*length)),3,(255,0,0))
        
        # add to writer
        writer.write(img)
except Exception as er:

    raise er.with_traceback(sys.exc_info()[2])
finally:
    # for index, row in df.iterrows():
    cap.release()
    writer.release()
    cv2.destroyAllWindows()

In [1]:
## RUN THIS SECTION AFTER SUPER COMPUTER FINISHES RUNNING  DlC ANALYSES.
from ezm_utils import run_open_probability_for_all_data
run_open_probability_for_all_data()

ALC_050319_1_control : ALC_050319_1_41G_EZM
ALC_050319_1_control : ALC_050319_1_41C_EZM
ALC_050319_1_control : ALC_050319_1_41R_EZM
ALC_050319_1_control : ALC_050319_1_43G_EZM
ALC_050319_1_control : ALC_050319_1_43B_EZM
ALC_050319_1_control : ALC_050319_1_43C_EZM
ALC_050319_1_control : ALC_050319_1_43R_EZM
ALC_050319_1_control : ALC_050319_1_42B_EZM
ALC_050319_1_control : ALC_050319_1_42C_EZM
ALC_050319_1_control : ALC_050319_1_42R_EZM
ALC_050319_1_control : ALC_050319_1_41B_EZM
ALC_050319_2_mC4 : ALC_050319_2_45R_EZM
ALC_050319_2_mC4 : ALC_050319_2_45B_EZM
ALC_050319_2_mC4 : ALC_050319_2_46C_EZM
ALC_050319_2_mC4 : ALC_050319_2_46B_EZM
ALC_050319_2_mC4 : ALC_050319_2_46R_EZM
ALC_050319_2_mC4 : ALC_050319_2_44R_EZM
ALC_050319_2_mC4 : ALC_050319_2_44B_EZM
ALC_050319_2_mC4 : ALC_050319_2_44C_EZM
ALC_051719_1_control : ALC_051719_1_42G_EZM
ALC_051719_1_control : ALC_051719_1_45C_EZM
ALC_051719_1_control : ALC_051719_1_45Bk_EZM
ALC_051719_1_control : ALC_051719_1_42Bk_EZM
ALC_051719_2_mC4 :